Transfer Learning Keras Implementation: Chimp Call Classification

In [1]:
#Import Dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os


In [2]:
#Parameters for Our Chimp Call Classifier
NUM_CLASSES = 11
CHANNELS = 3
IMAGE_RESIZE = 128
VGGISH_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

LOSS_METRICS = ['accuracy']

NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

BATCH_SIZE_TESTING = 1

VGGISH_CHECKPOINT_PATH = "./vggish/model_parameters/vggish_weights.ckpt"

In [3]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, Input, InputLayer
from vggish import vggish_keras

d:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#Preprocess audio and labels

In [5]:
#Run this at a later point in time after getting vggish right
feature_model = vggish_keras.get_vggish_keras()
feature_model.load_weights(VGGISH_CHECKPOINT_PATH)
for layer in feature_model.layers:
    layer.trainable = False
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 64, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 24, 16, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 24, 16, 256)       295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 24, 16, 256)       590080    
__________

In [6]:
feature_model.layers

In [15]:

top_model = Sequential()
#top_model.add(feature_model)
top_model.add(Dense(128,input_shape=feature_model.output_shape[1:]))
top_model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 11)                1419      
Total params: 17,931
Trainable params: 17,931
Non-trainable params: 0
_________________________________________________________________


In [16]:
from keras.layers.merge import concatenate

concatenated = concatenate([feature_model.output, Dense(128,input_shape=feature_model.output_shape[1:])])
out = Dense(1, activation='softmax', name='output_layer')(concatenated)


merged_model = Model([feature_model, top_model], out)
#merged_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
merged_model.summary()

ValueError: Layer concatenate_3 was called with an input that isn't a symbolic tensor. Received type: <class 'tensorflow.python.keras._impl.keras.layers.core.Dense'>. Full input: [<tf.Tensor 'fc2/Relu:0' shape=(?, 128) dtype=float32>, <tensorflow.python.keras._impl.keras.layers.core.Dense object at 0x000001D3CD67BF98>]. All inputs to the layer should be tensors.

In [ ]:

#top_model = feature_model
# top_model.add(feature_model)
top_model = Dense((None,  128), activation = 'relu')(feature_model)
top_model = Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION)(top_model)
top_model.summary()